## Random Forest Classifier With Pipeline And Hyperparameter Tuning

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [3]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [4]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [6]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [7]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [9]:
df.time.unique()

array([0, 1])

In [10]:
## independent and dependent feature
x = df.drop(labels=['time'],axis=1)
y = df.time

In [11]:
x.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [14]:
x['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [12]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,
                                                 random_state=42)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer  ## Handel Missing Values
from sklearn.preprocessing import StandardScaler  ## Feature scaling
from sklearn.preprocessing import OneHotEncoder   ## Categorical to Numerical
from sklearn.compose import ColumnTransformer

In [16]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill','tip','size']

In [18]:
## feature Engineering Automation (Create Pipeline)
## Numerical Pipelines
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),  ##Missing Values
        ('scaler',StandardScaler())   ## feature Scaling
    ]
)

#categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), ## Handel Missing Values
        ('onehotencoder',OneHotEncoder())  ## Categorical features to numerical
    ]
)

In [23]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [20]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [21]:
x_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [46]:
## Model Training Automation
models = {
    'Random Forest' : RandomForestClassifier(),
    'Logistic Regression' : LogisticRegression(),
    'Decision Tree' : DecisionTreeClassifier()
}

In [39]:
from sklearn.metrics import accuracy_score

In [44]:
def evaluate_model(x_train,y_train,x_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(x_train,y_train)

        # Predict Testing data
        y_test_pred = model.predict(x_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] = test_model_score

    return report

In [47]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [48]:
classifier = RandomForestClassifier()

In [49]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [50]:
from sklearn.model_selection import RandomizedSearchCV

In [51]:
cv = RandomizedSearchCV(classifier,param_distributions=params,
                   scoring='accuracy',cv=5,verbose=3)
cv.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=300;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=300;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.897 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=100;, score=0.923 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [52]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 10, 'criterion': 'gini'}

In [ ]:
## RandomForest Regression Solve it Internal Assignment

In [53]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
